![image](./images/movie-business-consumer-demand.jpg)

# **Movie Industry Analysis**
### Client  :  Microsoft
> *Authored by: Patrick Anastasio*

## Introduction

In preparation of entering the movie production business, Microsoft has asked me to prepare an analysis of current movie trends and to generate suggestions as to where to invest capital and how they can be sussesful in producing profitable movies. I will be analysing several datasets and making inferences off of financial information, ratings and popularity scores, as well as looking at established industry professionals to make suggestions on what genres and types of films to invest in and who to attach to projects to create buzz and generate an audience.

![image](./images/risky2.jpg)

## Business Problem

Movies are a 'risky business.' As a fledgling production house, Microsoft is unsure as to what kinds of movies to make, and where to invest capital. They lack the experience and industry knowledge that many of the top studios possess. Several factors go into producing a succesful movie. There are a few over-arching features that we will focus on: (1) gross revenue of the top rated and top grossing movies of the modern film era, (2) popular and highly rated genres, and (3) industry professionals who were instrumental in creating these movies.

## The Data

I have pulled in multiple datasets from three industry standard data aggregation sites.
- [Internet Movie Database (IMDB)](https://www.imdb.com/)
- [The Movie Database (TMDB)](https://www.themoviedb.org/?language=en-US)
- [The Numbers](https://www.the-numbers.com/)

I narrowed the scope of my analysis by initially filtering the data to only include movies made from 2010 forward. My subsequent filtering and analysis of these datasets focused on the following metrics:
> - Financials: 
>    - Budget and Domestic Gross Revenue
> - Ratings and Popularity Scores
> - Movie Genres
> - Names of directors, writers and actors/actresses

## The Method

After merging the datasets of interest I filtered out movies made prior to 2010. This constitutes the modern era of movie-making, and is characterized by new technologies and an explosion of investment.

I then converted data types as needed to allow me to operate on them. Specifically, converting objects to numbers to allow me to work with them mathematically.

From this merged and cleaned dataset I pulled dataframes based on:
1. ratings and popularity scores across all movies and averaged these into specific genres
2. domestic gross revenue across all movies, and then honing in on the top thirty (30) grossing movies and their budgets
3. directors of the top thirty (30) grossing movies, as well as writers and actors.

In [ ]:
# import the packages that will be used in this project

import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

## Import the data
#### Read in the raw data files, and create the dataframes I will work with

In [ ]:
names_by_id = pd.read_csv('data/zippedData/imdb.name.basics.csv.gz')

names_by_id.info()

In [ ]:
title_ratings = pd.read_csv('data/zippedData/imdb.title.ratings.csv.gz')

title_ratings.info()

In [ ]:
title_and_genre = pd.read_csv('data/zippedData/imdb.title.basics.csv.gz')

title_and_genre.info()

In [ ]:
directors_and_writers = pd.read_csv('data/zippedData/imdb.title.crew.csv.gz')

directors_and_writers.info()

In [ ]:
talent_list = pd.read_csv('data/zippedData/imdb.title.principals.csv.gz')

talent_list.info()

In [ ]:
popularity_and_votes = pd.read_csv('data/zippedData/tmdb.movies.csv.gz')

popularity_and_votes.info()

In [ ]:
budget_and_gross = pd.read_csv('data/zippedData/tn.movie_budgets.csv.gz')

budget_and_gross.info()

#### Merging the IMDB datasets

> These datasets contain titles, ratings, genres, production start year, and [above the line](https://www.amyclarkefilms.com/blog/above-and-below-the-line) names.

In [ ]:
# merge the IMDB dataframes with the common key 'tconst'

merged_df = title_and_genre.merge(
    title_ratings, on='tconst').merge(
    directors_and_writers, on='tconst').merge(
    talent_list, on='tconst')

merged_df.head()

In [ ]:
# the IMDB df 'names_by_id' does not have the key 'tconst'
# merge it on the 'nconst' key
merged_df = merged_df.merge(names_by_id, on='nconst')

merged_df.head()

#### Merging the remaining datasets

>The TMDB dataset contains information on popularity scores and audience voting.  
>The Numbers dataset contains information on the financials

In [ ]:
# merge the TMDB dataset with a "right" merge, since we only want to keep titles that have a popularity score and votes
# the common key is 'original_title'

merged_df = merged_df.merge(popularity_and_votes, on='original_title', how='right')

merged_df.head()

In [ ]:
# The Numbers dataset does not contain a common key so we will have to make one

budget_and_gross.head()

In [ ]:
# notice the 'movie' key is the same as the 'original_title' key
# rename the 'movie' key to 'original_title' to merge it

budget_and_gross.rename({'movie':'original_title'}, axis=1, inplace=True)

budget_and_gross.head(1)

In [ ]:
# merge The Numbers dataset with a "right" merge to keep only titles with financial info

merged_df = merged_df.merge(budget_and_gross, on='original_title', how='right')

merged_df.head()

In [ ]:
# take a look at the completely merged dataframe

merged_df.info()

## Clean the data


#### As stated above, the focus is on movies produced in the modern era  
>I need to filter out titles that were made before the year 2010

In [ ]:
# notice from the info that 'start_year' is of dtype: float, which will make filtering easier that converting a dtype: object to datetime
# filter on movies produced from 2010 forward

merged_df = merged_df[merged_df['start_year'] >= 2010.00]

merged_df['start_year'].min()

#### Unfortunately, the values in the financial columns are of dtype: object
>I need to convert these values to a number dtype to work with them mathematically

In [ ]:
# create a function that will take an object/value and make it into a number

def drop_dollar_sign_and_commas(value):
    """
    this will split the object into a list of characters using the list() function
    then iterate over the list and drop the $ sign, and remove commas from the list
    use the .remove() method to drop the $
    use a for loop to remove the commas, as .remove() will only remove the first instance, and some values contain more than one comma
    then use the .join() method to reconneect the list into a single string
    finally turn that string into a float, and return it
    """
    
    value_list = list(value)
    value_list.remove('$')
    for char in value_list:
        if ',' == char:
            value_list.remove(char)
    value_float = float(''.join(value_list))
    return value_float

In [ ]:
# create new columns for the float values using .map() and the function above

merged_df['Budget'] = merged_df['production_budget'].map(drop_dollar_sign_and_commas)
merged_df['Domestic Gross'] = merged_df['domestic_gross'].map(drop_dollar_sign_and_commas)

# and check it
merged_df.info()

In [ ]:
# drop columns that I will not need

merged_df.drop(['primary_title', 'runtime_minutes', 'job', 'birth_year', 'death_year', 'primary_profession', 'known_for_titles', 'Unnamed: 0', 'genre_ids', 'id_x', 'original_language', 'release_date_x', 'title', 'id_y', 'release_date_y', 'production_budget', 'domestic_gross', 'worldwide_gross'], axis=1, inplace=True)

merged_df.info()

## Analyze the data
#### I am looking to draw inferences from
* a movies popularity measured by its ratings
* budget and gross (for simplicity's sake I will only focus on domestic gross)
* talent attached to popular and profitable movies

#### Genre by average rating
>I pull out a separate dataframe that ony contains information on movie ratings, popularity, and the financials

In [ ]:
# pull the desired slice of the dataframe

genre_by_rating = merged_df.loc[:,('original_title', 'genres', 'averagerating', 'numvotes', 'Budget', 'Domestic Gross')]

genre_by_rating['genres'].info()

In [ ]:
# drop the null values from 'genres'

genre_by_rating.dropna(subset=['genres'], axis=0, inplace=True)

genre_by_rating.info()

In [ ]:
# notice that there are 'genres' values that have multiple genres listed separated by commas
# I will focus on movies with only one genre
# drop values with multiple genres

genre_by_rating_multigenre = genre_by_rating[genre_by_rating['genres'].str.contains(',')].index
genre_by_rating.drop(genre_by_rating_multigenre, inplace=True)

genre_by_rating.info()

In [ ]:
# there are also many duplicates
# drop duplicate titles

genre_by_rating.drop_duplicates(subset='original_title', inplace=True)

genre_by_rating.head()